In [17]:
from glob import glob
import uproot as ur
import numpy as np
import json
import os.path as osp
from phc import module_reload
module_reload('zhh')
from zhh import parse_sample_path, get_preselection_passes, ProcessIndex
version = 'v1'

REPO_ROOT = '/afs/desy.de/user/b/bliewert/public/MarlinWorkdirs/ZHH'
DATA_ROOT = f'/nfs/dust/ilc/user/bliewert/zhh/PreselectionFinal/{version}'
INDEX_DIR = '/nfs/dust/ilc/user/bliewert/zhh/CreateRawIndex/v1'

processes = np.load(f'{INDEX_DIR}/processes.npy')
samples = np.load(f'{INDEX_DIR}/samples.npy')

Introduce indices on samples and processes (included in future runs)

In [12]:
dtype_sample = ProcessIndex.dtype_sample
dtype_process = ProcessIndex.dtype_process

if not 'sid' in samples.dtype.names:
    samples_new = np.empty(len(samples), dtype=dtype_sample)
    samples_new['sid'] = np.arange(len(samples))
    for col in samples.dtype.names:
        samples_new[col] = samples[col]
        
    if len(samples) == len(samples_new):
        np.save(f'{INDEX_DIR}/samples.npy', samples_new)

if not 'pid' in processes.dtype.names:
    #processes_new = np.array([pids, *(processes[col] for col in processes.dtype.names)], dtype=dtype_process)
    processes_new = np.empty(len(processes), dtype=dtype_process)
    processes_new['pid'] = np.arange(len(processes))
    for col in processes.dtype.names:
        processes_new[col] = processes[col]

    if len(processes) == len(processes_new):
        np.save(f'{INDEX_DIR}/processes.npy', processes_new)
        
print('Conversion successful')

Conversion successful


Testing

In [43]:
rf = ur.open('/afs/desy.de/user/b/bliewert/public/MarlinWorkdirs/ZHH/output/zhh_PreSelection_llHH.root')

In [55]:
tree = rf['eventTree']
rf_keys = tree.keys()

rf['eventTree'][rf_keys[0]].array()

<Array [403001] type='1 * int32'>

Prototyping

In [18]:
results = get_preselection_passes(DATA_ROOT)

  0%|          | 0/14449 [00:00<?, ?it/s]

/afs/desy.de/user/b/bliewert/public/MarlinWorkdirs/ZHH/zhh/analysis/PreselectionAnalysis.py:163: RuntimeWarning: invalid value encountered in scalar divide
  return process_sigma_fb * 1000 *lum_inv_ab * w_pol / n_gen


In [27]:
processes

array([(  0, '2f_z_bhabhang', '2f_z_bhabhang_LR', -1,  1, 1.33070797e+05, 7.84000015e+01, 250127),
       (  1, 'qqhh', 'qqhh_RL',  1, -1, 1.44522369e-01, 1.02380312e-04, 403010),
       (  2, 'e3e3hh', 'e3e3hh_LR', -1,  1, 1.11406688e-02, 5.84571035e-06, 403003),
       (  3, 'n23n23qqh', 'n23n23qqh_LR', -1,  1, 2.59607166e-01, 3.96785181e-04, 403017),
       (  4, '2f_z_nung', '2f_z_nung_LR', -1,  1, 2.80930996e+04, 3.27000008e+01, 250130),
       (  5, 'n23n23qqh', 'n23n23qqh_RL',  1, -1, 1.10307105e-01, 1.66818631e-04, 403018),
       (  6, 'yyvllv', 'yyvllv_LR', -1,  1, 7.87087555e+01, 7.24000037e-02, 108665),
       (  7, '4f_lowmee_sze_l', '4f_lowmee_sze_l_LR', -1,  1, 5.46110010e+00, 2.60000005e-02, 250119),
       (  8, 'yyveyx', 'yyveyx_LR', -1,  1, 1.17086838e+02, 8.83999988e-02, 108668),
       (  9, '2f_z_h', '2f_z_h_LR', -1,  1, 3.24705000e+04, 8.88000031e+01, 250114),
       ( 10, 'xxxyyx', 'xxxyyx_LR', -1,  1, 1.57004671e+01, 6.85999990e-02, 108687),
       ( 11, 'yycyy

In [25]:
np.save(f'{REPO_ROOT}/preselection.npy', results)
np.save(f'{REPO_ROOT}/samples.npy', samples)
np.save(f'{REPO_ROOT}/processes.npy', processes)

In [22]:
for entry in results:
    print(f'{entry["proc_pol"]} | {entry["n_gen"]} events | wt: {entry["weight"]} | {entry["n_pass_llhh"]} : {entry["n_pass_vvhh"]} : {entry["n_pass_qqhh"]}')
    
#np.save(f'{REPO_ROOT}/results.npy', results)

2f_z_bhabhag_LL | 74267 events | wt: 50.25239562988281 | 310 : 0 : 0
2f_z_bhabhag_LR | 169220 events | wt: 50.405826568603516 | 659 : 0 : 0
2f_z_bhabhag_RL | 9393 events | wt: 50.260196685791016 | 41 : 0 : 0
2f_z_bhabhag_RR | 15298 events | wt: 50.2548828125 | 66 : 0 : 0
2f_z_bhabhang_LL | 2966291 events | wt: 52.63407516479492 | 3252 : 0 : 0
2f_z_bhabhang_LR | 2984402 events | wt: 104.33770751953125 | 3772 : 0 : 0
2f_z_bhabhang_RL | 362833 events | wt: 50.25137710571289 | 421 : 0 : 0
2f_z_bhabhang_RR | 641141 events | wt: 50.251487731933594 | 679 : 0 : 0
2f_z_h_LR | 1498124 events | wt: 50.717411041259766 | 94 : 492 : 2956
2f_z_h_RL | 50131 events | wt: 50.25349426269531 | 2 : 16 : 95
2f_z_l_LR | 259192 events | wt: 50.083866119384766 | 100 : 3 : 6
2f_z_l_RL | 12144 events | wt: 50.08896255493164 | 1 : 0 : 0
2f_z_nung_LR | 1313442 events | wt: 50.0500602722168 | 0 : 0 : 0
2f_z_nung_RL | 5415 events | wt: 50.09492111206055 | 0 : 0 : 0
4f_lowmee_sze_l_LL | 8 events | wt: 61.729759216308